# 04 - Grover Optimizer

#### Uncomment the cell below to pip install the necessary modules if not already installed

#### Note: Works with Qiskit Version 1.4.1 and Quantum Rings Qiskit Toolkit Version 0.1.10

In [2]:
# %pip install qiskit-algorithms
# %pip install quantumrings-toolkit-qiskit==0.1.10
# %pip install qiskit==1.4.1
# %pip install qiskit-optimization

#### Restart the kernel after installing any of the missing packages

### Introduction

In this notebook, we will explore the [Grover Adaptive Search (GAS)](https://arxiv.org/abs/1912.04088) which is an advancement on the original [Grover's Search Algorithm](https://arxiv.org/abs/quant-ph/9605043) developed by Gilliam, Woerner, and Gonciulea (2019). 

The Grover Adaptive Search has been explored as a possible solution for combinatorial optimization problems, alongside variational algorithms such as Variational Quantum Eigensolver (VQE) and Quantum Approximate Optimization Algorithm (QAOA). The algorithm iteratively applies Grover Search to find the optimum value of an objective function by using the best-known value from the previous run as a threshold. The adaptive oracle used in GAS recognizes all values above or below the current threshold (for max and min respectively), decreasing the size of the search space every iteration the threshold is updated, until an optimum is found.

We will explore each component of the `GroverOptimizer`, which utilizes the techniques described in GAS, by minimizing a Quadratic Unconstrained Binary Optimization (QUBO) problem.

### Grover Adaptive Search

Let us cover some of the theory behind the Grover Adaptive Search before we jump in to the coding. 

The Grover Search, at its core, needs three ingredients:

1. A state preparation operator $A$ to construct a superposition of all states in the search space.
2. An oracle operator $O$ that recognizes the states of interest and multiplies their amplitudes by -1.
3. The Grover diffusion operator $D$ that multiplies the amplitude of the $\ket{0}_n$ state by -1.

While implementations of GAS vary around the specific use case, the general framework still loosely follows the steps described below.

<div>
<img src="aux_files/GAS-Algorithm.png" width="400"/>
</div>

`GroverOptimizer` uses `QuadraticProgramToNegativeValueOracle` to construct $A_y$ such that it prepares a $n$-qubit register to represent the equal superposition of all $\ket{x}_n$ and a $m$-qubit register to (approximately) represent the corresponding $\ket{Q(x)-y}_m$. All states with ($Q(x)-y$) negative should be flagged by $O_y$. Note that in the implementation discussed, the oracle operator is actually independent of $y$, but this is not a requirement. For clarity, we will refer to the oracle as $O$ when the oracle is independent of $y$.

Formally, `QuadraticProgramToNegativeValueOracle` constructs an $A_y$ and $O$ such that:

<div>
<img src="aux_files/a-o-dfn.png" width="400"/>
</div>

where $\ket{x}$ is the binary encoding of the integer $x$.

At each iteration in which the threshold $y$ is updated, we adapt $A_y$ such that the function values are shifted up or down (for minimum and maximum respectively) by $y$. For example, in the context of finding the minimum, as the value of $y$ decreases, the search space (negative values) also decreases, until only the minimum value remains. A concrete example will be explored in the next section.

### Find the Minimum of a QUBO Problem using GroverOptimizer

The following is an example of a minimization problem

\begin{eqnarray}
\min_{x \in \{0, 1\}^3} -2x_0x_2 - x_1x_2 - 1x_0 + 2x_1 - 3x_2.
\end{eqnarray}

For our initial steps, we create a `docplex` model that defines the problem above, and then use the `from_docplex_mp()` function to convert the model to a `QuadraticProgram`, which can be used to represent a QUBO in Qiskit Optimization.

In [1]:
# This code is from the tutorial at:
# https://qiskit-community.github.io/qiskit-optimization/tutorials/04_grover_optimizer.html

In [2]:
from qiskit_algorithms import NumPyMinimumEigensolver

from quantumrings.toolkit.qiskit import QrSamplerV1 as Sampler

from qiskit_optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model

In [3]:
model = Model()
x0 = model.binary_var(name="x0")
x1 = model.binary_var(name="x1")
x2 = model.binary_var(name="x2")
model.minimize(-x0 + 2 * x1 - 3 * x2 - 2 * x0 * x2 - 1 * x1 * x2)
qp = from_docplex_mp(model)
print(qp.prettyprint())

Problem name: docplex_model1

Minimize
  -2*x0*x2 - x1*x2 - x0 + 2*x1 - 3*x2

Subject to
  No constraints

  Binary variables (3)
    x0 x1 x2



We create a `GroverOptimizer` that uses 6 qubits to encode the value, and will terminate after there have been 10 iterations of GAS without progress (i.e. the value of $y$ does not change). The `solve()` function takes the `QuadraticProgram` we created earlier, and returns a results object that contains information about the run.

In [4]:
grover_optimizer = GroverOptimizer(6, num_iterations=10, sampler=Sampler())
results = grover_optimizer.solve(qp)
print(results.prettyprint())

objective function value: -6.0
variable values: x0=1.0, x1=0.0, x2=1.0
status: SUCCESS


This results in the optimal solution $x_0=1$, $x_1=0$, $x_2=1$ and the optimal objective value of $-6$ (most of the time, after all, it is a randomized algorithm).

A custom visualization of the quantum state shows a possible run of `GroverOptimizer` applied to this QUBO.

<div>
<img src="aux_files/GAS-QUBO-Example.png" width="800"/>
</div>

Each graph shows a single iteration of GAS, with the current values of $r$ (= iteration counter) and $y$ (= threshold/offset) shown in the title. The X-axis displays the integer equivalent of the input (e.g. '101' $\rightarrow$ 5), and the Y-axis shows the possible function values. As there are 3 binary variables, there are $2^3=8$ possible solutions, which are shown in each graph. The color intensity indicates the probability of measuring a certain result (with bright intensity being the highest), while the actual color indicates the corresponding phase (see the phase color-wheel below). Note that as $y$ decreases, we shift all of the values up by that amount, meaning there are fewer and fewer negative values in the distribution, until only one remains (the minimum).

<div>
<img src="aux_files/color-wheel-phase.png" width="200"/>
</div>

### Check that the GroverOptimizer finds the correct value

We can verify that the algorithm is working correctly using the `MinimumEigenOptimizer` in Qiskit Optimization

In [5]:
exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
exact_result = exact_solver.solve(qp)
print(exact_result.prettyprint())

objective function value: -6.0
variable values: x0=1.0, x1=0.0, x2=1.0
status: SUCCESS


### Conclusion

Congratulations! You have demonstrated the Grover Adaptive Search algorithm on a small optimization example! Feel free to explore the [paper](https://arxiv.org/pdf/1912.04088) linked here and at the top for a more in depth understanding.